In [62]:
import pandas as pd
pd.set_option('display.max_rows',500)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
import pandas_datareader.data as pdr
from pandas_datareader.data import DataReader
import yfinance as yf

# For time stamps
import datetime

from prophet import Prophet

In [63]:
# download stocks
file = pd.read_csv('M6_Universe.csv')
stocks_list = file['symbol'].values.tolist()

#end = datetime.datetime.now()
end = datetime.datetime(2022,9,18)
start = datetime.datetime(end.year - 3, end.month, end.day)

last_month_days = []
for year in range(start.year, end.year+1):
    for month in range(1,13):
        first_day = datetime.datetime(year, month, 1)
        prev_day = first_day - datetime.timedelta(days=1)
        last_month_days += [prev_day.strftime('%Y-%m-%d')]

## Download 100 assets

In [64]:
all_stocks = DataReader(stocks_list, 'yahoo', start, end)['Adj Close']

all_stocks.describe()

Symbols,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
count,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,...,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000
mean,104.908084,262.144230,84.446521,139.426049,113.717511,96.092824,212.028597,141.219907,189.359753,159.905709,...,30.921476,113.666138,49.709738,153.893649,86.736765,63.691508,62.251846,64.998256,70.220936,62.451753
std,27.103004,61.301502,8.074330,25.488422,14.569241,35.597346,63.964842,29.598877,33.861642,39.667935,...,6.007736,16.208469,15.840490,27.969647,13.867947,11.498571,6.354460,7.165775,13.209813,45.872051
min,57.469627,138.960983,64.298126,76.798347,76.101143,37.084087,78.741386,83.830498,110.256210,75.713036,...,16.813219,71.814064,20.812737,85.449234,46.945175,39.308064,41.590984,45.549580,36.500797,17.629999
25%,81.983767,205.549877,78.937849,119.837843,100.269836,59.581530,152.064312,116.085001,156.147820,122.937513,...,26.047762,98.995071,35.970816,127.705189,76.137398,54.373772,57.703673,58.840935,57.210014,24.982500
50%,101.580025,261.400803,83.154476,138.107567,112.175915,99.060841,222.065048,154.467499,193.478516,164.870628,...,31.667564,114.102394,48.301079,156.336731,87.970112,61.083839,62.041058,64.447361,73.721546,50.760000
75%,126.043356,312.494553,87.717470,159.097713,127.072527,131.340664,268.208122,164.633247,216.122528,198.866158,...,36.412895,129.166191,55.405662,176.385250,99.499004,75.179567,66.163227,70.927135,81.898720,88.599998
max,171.877823,411.651428,105.180000,191.601440,145.579788,165.868652,324.398621,186.570496,252.427414,223.893570,...,40.825043,141.664886,90.272392,209.854355,105.722588,85.122887,77.594452,79.452660,89.421799,276.000000


In [65]:
# extract monthly returns
month_stocks = all_stocks.copy()
month_stocks['Datestring'] = month_stocks.index
    
month_stocks['year'] = pd.DatetimeIndex(month_stocks.index).year
month_stocks['month'] = pd.DatetimeIndex(month_stocks.index).month

month_on_month = month_stocks.sort_values(by='Date').groupby(['year','month']).last()
month_on_month = month_on_month.drop(month_on_month.tail(1).index).set_index(['Datestring'])

monthly_returns = month_on_month.pct_change().round(3).T
monthly_returns

Datestring,2019-09-30,2019-10-31,2019-11-29,2019-12-31,2020-01-31,2020-02-28,2020-03-31,2020-04-30,2020-05-29,2020-06-30,...,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-29,2022-05-31,2022-06-30,2022-07-29,2022-08-31
Symbols,,,,,,,,,,,,,,,,,,,,,
ABBV,NaN,0.066,0.103,0.009,-0.073,0.058,-0.111,0.095,0.127,0.059,...,0.005,0.175,0.021,0.079,0.097,-0.086,0.003,0.039,-0.054,-0.063
ACN,NaN,-0.032,0.085,0.047,-0.022,-0.120,-0.096,0.139,0.089,0.065,...,-0.004,0.160,-0.145,-0.106,0.067,-0.107,-0.006,-0.070,0.107,-0.058
AEP,NaN,0.007,-0.025,0.035,0.103,-0.138,-0.104,0.039,0.035,-0.066,...,-0.034,0.098,0.016,0.012,0.101,-0.007,0.038,-0.060,0.027,0.025
AIZ,NaN,0.002,0.059,-0.013,-0.004,-0.072,-0.137,0.021,-0.028,0.007,...,-0.053,0.025,-0.021,0.117,0.071,0.000,-0.025,-0.022,0.017,-0.095
ALLE,NaN,0.120,0.034,0.040,0.038,-0.111,-0.197,0.093,-0.008,0.028,...,-0.036,0.074,-0.073,-0.067,-0.038,0.041,-0.023,-0.120,0.080,-0.100
AMAT,NaN,0.087,0.071,0.054,-0.050,0.005,-0.212,0.084,0.135,0.076,...,0.079,0.069,-0.122,-0.027,-0.018,-0.163,0.065,-0.224,0.165,-0.110
AMP,NaN,0.026,0.093,0.017,-0.007,-0.141,-0.275,0.122,0.229,0.071,...,-0.038,0.042,0.009,-0.011,0.002,-0.116,0.045,-0.140,0.136,-0.002
AMZN,NaN,0.023,0.014,0.026,0.087,-0.062,0.035,0.269,-0.013,0.130,...,0.040,-0.049,-0.103,0.027,0.061,-0.238,-0.033,-0.116,0.271,-0.061
AVB,NaN,0.011,-0.015,-0.015,0.033,-0.074,-0.259,0.107,-0.043,0.002,...,0.009,0.064,-0.033,-0.023,0.047,-0.084,-0.086,-0.058,0.101,-0.061


In [66]:
# Generate monthly return forecast table
future_steps = 4
first_time = True
for ticker in stocks_list:
    print('Ticker: %s...' % ticker)
    df = monthly_returns.T[ticker].dropna().reset_index().rename(columns={'Datestring':'ds', ticker:'y'})
    model = Prophet(interval_width=0.95).fit(df)
    future_dates = model.make_future_dataframe(periods=future_steps, freq='M')
    forecast = model.predict(future_dates)
    if first_time:
        forecast_df = forecast[['ds']].set_index('ds')
        lower_df = forecast_df.copy()
        yhat_df = forecast_df.copy()
        upper_df = forecast_df.copy()
        first_time = False

    forecast = forecast.set_index('ds')

    lower_df[ticker] = forecast['yhat_lower']
    yhat_df[ticker] = forecast['yhat']
    upper_df[ticker] = forecast['yhat_upper']
        

16:05:14 - cmdstanpy - INFO - Chain [1] start processing


Ticker: ABBV...


16:05:14 - cmdstanpy - INFO - Chain [1] done processing
16:05:14 - cmdstanpy - INFO - Chain [1] start processing


Ticker: ACN...


16:05:14 - cmdstanpy - INFO - Chain [1] done processing
16:05:15 - cmdstanpy - INFO - Chain [1] start processing


...

16:06:18 - cmdstanpy - INFO - Chain [1] done processing
16:06:18 - cmdstanpy - INFO - Chain [1] start processing


Ticker: XLB...


16:06:18 - cmdstanpy - INFO - Chain [1] done processing
16:06:19 - cmdstanpy - INFO - Chain [1] start processing


Ticker: VXX...


16:06:19 - cmdstanpy - INFO - Chain [1] done processing


In [67]:
lower_df

,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
ds,,,,,,,,,,,,,,,,,,,,,
2019-10-31,-0.068937,-0.107468,-0.030852,-0.078941,-0.029429,-0.054056,-0.076060,-0.069896,-0.132845,0.021889,...,-0.082698,-0.037852,-0.196197,-0.076105,-0.085321,-0.034054,-0.049557,-0.073458,-0.076405,-0.379852
2019-11-29,-0.001743,-0.020601,-0.099641,-0.036959,-0.082851,-0.089833,-0.057583,-0.078221,-0.138141,-0.067991,...,-0.072039,-0.009904,-0.229088,-0.069754,-0.059167,-0.035847,-0.086844,-0.039575,-0.064851,-0.427700
2019-12-31,-0.056293,-0.058158,-0.036807,-0.114984,-0.062894,-0.131917,-0.161161,-0.084275,-0.142183,-0.090466,...,-0.107234,-0.013947,-0.233954,-0.089983,-0.117948,-0.059285,-0.028503,-0.015844,-0.077015,-0.325699
2020-01-31,-0.104740,-0.128354,-0.049703,-0.091357,-0.121168,-0.114845,-0.068524,-0.049062,-0.095251,-0.097040,...,-0.089146,-0.072646,-0.139302,-0.070060,-0.109846,-0.047311,-0.052246,-0.059956,-0.106529,-0.196669
2020-02-28,-0.019857,-0.126646,-0.144317,-0.073470,-0.152183,-0.020977,-0.125195,-0.045722,-0.123389,-0.148079,...,-0.110678,-0.073063,-0.181790,-0.068497,-0.096875,-0.062842,-0.120445,-0.087579,-0.083467,-0.170248
2020-03-31,-0.156867,-0.082694,-0.090422,-0.108961,-0.146482,-0.176964,-0.253508,-0.103549,-0.194757,-0.088096,...,-0.188703,-0.043094,-0.387967,-0.126913,-0.152123,-0.120079,-0.067366,-0.053481,-0.110684,0.064445
2020-04-30,0.001991,0.052403,-0.026352,-0.006661,-0.000332,-0.040374,0.039906,0.155871,0.008557,0.060365,...,0.012183,0.052246,0.013715,0.099503,0.012633,0.083998,-0.009414,-0.002187,0.059337,-0.536039
2020-05-29,-0.033300,-0.029724,-0.071930,-0.093940,-0.065109,0.019415,0.008955,-0.074871,-0.109213,-0.068523,...,-0.049749,-0.019837,-0.124921,-0.037524,-0.046504,0.001194,-0.050146,-0.048807,-0.007756,-0.467028
2020-06-30,-0.036966,-0.012978,-0.122172,-0.099105,-0.068255,-0.031794,-0.048096,0.033126,-0.089028,-0.056241,...,-0.102943,-0.058875,-0.174105,-0.015144,-0.070144,-0.038521,-0.094174,-0.049902,-0.072270,-0.448197


In [68]:
# Quantile table generation
timestamp = '2022-10-31'
quantile_table = pd.DataFrame([],columns=stocks_list).T
quantile_table['5'] = np.nan
quantile_table['y_hat'] = np.nan
quantile_table['95'] = np.nan

quantile_table = quantile_table.T

for ticker in stocks_list:
    q_5 = lower_df[ticker][timestamp]
    q_y = yhat_df[ticker][timestamp]
    q_95 = upper_df[ticker][timestamp]

    quantile_table[ticker] = [q_5, q_y, q_95]
    
    
quantile_table = quantile_table.T

# 95th percentile is 3 std away from mean
quantile_table['f_std'] = (quantile_table['95'] - quantile_table['y_hat']) / 3

quantile_table

,5,y_hat,95,f_std
ABBV,-0.089169,0.000187,0.090487,0.030100
ACN,-0.233824,-0.155475,-0.070908,0.028189
AEP,0.010244,0.088738,0.162448,0.024570
AIZ,-0.085886,0.000710,0.089759,0.029683
ALLE,-0.084201,0.019514,0.121020,0.033835
AMAT,-0.207223,-0.084024,0.037821,0.040615
AMP,-0.170574,-0.054142,0.060689,0.038277
AMZN,-0.269170,-0.177762,-0.088072,0.029897
AVB,-0.137544,-0.036791,0.054725,0.030506
AVY,-0.020641,0.072740,0.166948,0.031402


In [69]:
# generate table of sampling
sampling_table = quantile_table.copy().drop(quantile_table.columns, axis=1)
for i in range(1,101):
    sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
    
sampling_table = sampling_table.round(4)
sampling_table

,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10,...,s_91,s_92,s_93,s_94,s_95,s_96,s_97,s_98,s_99,s_100
ABBV,-0.0121,0.0054,0.0078,-0.0337,0.0203,0.0245,-0.0079,-0.0021,-0.0122,-0.0947,...,-0.0043,0.0285,-0.0370,0.0167,0.0259,0.0016,0.0489,-0.0395,-0.0187,-0.0167
ACN,-0.1472,-0.1740,-0.1459,-0.1248,-0.1369,-0.1255,-0.1431,-0.1992,-0.1928,-0.1464,...,-0.1675,-0.1973,-0.1391,-0.2119,-0.1421,-0.1630,-0.1868,-0.1385,-0.1334,-0.1561
AEP,0.1263,0.0586,0.0738,0.0702,0.1008,0.0353,0.0680,0.0946,0.1135,0.1028,...,0.1226,0.0941,0.0807,0.0849,0.0968,0.1107,0.0662,0.1109,0.0478,0.0730
AIZ,0.0349,-0.0071,-0.0119,-0.0278,0.0730,-0.0002,-0.0002,0.0043,0.0074,-0.0197,...,0.0408,-0.0715,0.0232,-0.0800,-0.0625,0.0252,-0.0010,0.0497,-0.0322,-0.0293
ALLE,0.0140,-0.0502,0.0130,-0.0221,0.0189,0.0069,0.0020,0.0245,-0.0092,0.0380,...,0.0161,0.0177,0.0232,0.0028,0.0412,0.0229,0.0042,0.0396,0.0265,0.0281
AMAT,-0.1796,-0.1263,-0.0939,-0.1338,-0.0752,-0.0053,-0.1115,-0.0944,-0.1150,-0.0745,...,-0.0765,-0.0585,-0.1004,-0.1035,-0.0131,-0.0954,-0.0964,-0.1174,-0.1327,-0.0654
AMP,-0.0921,-0.0953,-0.0695,-0.0414,-0.0747,-0.0369,-0.0569,0.0119,-0.0404,-0.0858,...,-0.0099,-0.0592,-0.0949,-0.0412,-0.0802,-0.0991,-0.1127,-0.0422,-0.0575,-0.0471
AMZN,-0.1455,-0.2054,-0.1888,-0.2409,-0.2560,-0.1901,-0.1570,-0.1699,-0.1356,-0.1789,...,-0.1623,-0.1207,-0.2302,-0.1411,-0.1289,-0.1763,-0.1788,-0.1465,-0.1986,-0.1513
AVB,-0.0666,-0.0010,-0.0081,-0.0205,-0.0251,-0.0199,-0.1019,-0.0446,-0.0571,0.0114,...,-0.0264,-0.0593,-0.0355,0.0130,-0.0127,-0.0388,-0.0605,-0.0652,-0.0627,-0.0729
AVY,0.0422,0.0515,0.0649,0.0912,0.0609,0.1081,0.0509,0.0473,0.0862,0.0726,...,0.1038,0.0953,0.0763,0.0093,0.0591,0.0235,0.0441,0.0313,0.0453,0.0687


In [70]:
# extract total ranks
def generate_rank(df):
    ranks = pd.DataFrame(index=df.index, columns=df.columns)
    vector_ranks = pd.DataFrame(index=df.index, columns=df.columns)
    
    for col in df.columns:
        if df[col].isnull().all():
            continue
        ranks[col], vector_ranks[col] = generate_rank_from_values(df[col].values)
        
    return ranks, vector_ranks

def generate_rank_from_values(value_list):
    if len(value_list) != 100:
        print('Values list not 100 in number')
        return
    df = pd.DataFrame(value_list, columns=['data'])
    orig_index = df.index
    #df[241] = [random.randint(1,20) for i in range(100)]
    df = df.sort_values('data',ascending=False)
    df['my100rank'] = range(100,0,-1)
    df['my5rank'] = [5]*20 + [4]*20 + [3]*20 + [2]*20 + [1]*20
    df['pyrank'] = df['data'].rank()
    df['5rank'] = pd.qcut(df['data'], 5, labels=False, precision=1) + 1
    df['diff'] = df['my5rank'] != df['5rank']
    uniq_ranks = df[df['diff'] == True]['pyrank'].unique()
    df['adjusted5rank'] = df['my5rank']
    df['rankvector'] = np.nan
    
    df = df.join(pd.get_dummies(df['my5rank']))
    
    
    # ties on the margins of the classes
    for rank in uniq_ranks:
        rank_index = df[df['pyrank']==rank].index
        population = df['my5rank'][rank_index]
        df.loc[rank_index,'adjusted5rank'] = round(population.mean(),2)
        for p in population:
            df.loc[rank_index, p] = round(sum(population == p) / len(population),2)
            
    
    
    #print(df.to_string())
    ranks = df['adjusted5rank'][orig_index].values.tolist()
    vector_ranks = []
    for index, row in df.loc[orig_index].iterrows():
        vector_ranks.append([row[1],row[2],row[3],row[4],row[5]])
    
    # clean house
    for p in [1,2,3,4,5]:
        df[f'Rank {p}'] = df[p]
    df = df.drop(columns=[1,2,3,4,5])
    return ranks, vector_ranks

ranks, vector_ranks = generate_rank(sampling_table)
ranks

,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10,...,s_91,s_92,s_93,s_94,s_95,s_96,s_97,s_98,s_99,s_100
ABBV,4.0,5,4.0,3,5,5,4,4,4,2.0,...,4,5.0,3.0,5,5,4,5,3,4,4
ACN,1.0,1,1.0,1,1,1,1,1,1,1.0,...,1,1.0,1.0,1,1,1,1,1,1,1
AEP,5.0,5,5.0,5,5,5,5,5,5,5.0,...,5,5.0,5.0,5,5,5,5,5,5,5
AIZ,5.0,4,4.0,3,5,4,4,4,4,4.0,...,5,2.0,4.5,2,2,5,4,5,3,3
ALLE,4.0,3,4.0,4,5,4,4,5,4,5.0,...,4,4.0,4.5,4,5,4,4,5,5,5
AMAT,1.0,1,2.0,1,2,4,1,2,1,2.0,...,2,2.0,1.0,1,4,1,2,1,1,2
AMP,2.0,2,2.0,3,2,3,2,4,3,2.0,...,4,2.0,2.0,3,2,1,1,3,3,3
AMZN,1.0,1,1.0,1,1,1,1,1,1,1.0,...,1,1.0,1.0,1,1,1,1,1,1,1
AVB,2.0,4,4.0,4,3,3,2,3,2,4.0,...,3,2.0,3.0,4,4,3,2,2,2,2
AVY,5.0,5,5.0,5,5,5,5,5,5,5.0,...,5,5.0,5.0,4,5,5,5,5,5,5


In [71]:
pmf_vranks = []

for row in ranks.index:
    pmf, bins = np.histogram(ranks.T[row].dropna().values - 0.001, bins = range(0,6), density=True)
    pmf = np.round(pmf, 3).tolist()
    pmf_vranks.append(pmf)
    

ranks_table = ranks.copy().drop(ranks.columns, axis=1)

# rank histogram per instrument
ranks_table['PMF Rank Vector'] = pmf_vranks
# expected rank
ranks_table['PMF Rank'] = np.dot(np.array(pmf_vranks), np.array(range(1,6)))
ranks_table

,PMF Rank Vector,PMF Rank
ABBV,"[0.0, 0.04, 0.2, 0.42, 0.34]",4.06
ACN,"[1.0, 0.0, 0.0, 0.0, 0.0]",1.00
AEP,"[0.0, 0.0, 0.0, 0.0, 1.0]",5.00
AIZ,"[0.0, 0.05, 0.13, 0.51, 0.31]",4.08
ALLE,"[0.0, 0.03, 0.08, 0.38, 0.51]",4.37
AMAT,"[0.38, 0.41, 0.1, 0.1, 0.01]",1.95
AMP,"[0.16, 0.4, 0.28, 0.16, 0.0]",2.44
AMZN,"[1.0, 0.0, 0.0, 0.0, 0.0]",1.00
AVB,"[0.03, 0.24, 0.4, 0.29, 0.04]",3.07
AVY,"[0.0, 0.0, 0.0, 0.02, 0.98]",4.98


In [72]:
# break up single column vector array to separate columns
pmf_values = np.array([x for x in ranks_table['PMF Rank Vector'].values])
submission = ranks_table.filter('PMF Rank Vector')
for i in range(5):
    submission[f'Rank {i+1}'] = pmf_values[:,i]

    
diff = 1 - submission[['Rank 1','Rank 2','Rank 3','Rank 4','Rank 5']].sum(1)
submission['Rank 5'] = submission['Rank 5'] + diff

# portfolio strategy
submission['Decision'] = submission['Rank 5'] / submission['Rank 5'].sum()
# round up to 5 dp
submission = submission.round(5)

diff = 1 - submission['Decision'].sum()
submission['Decision'][-1] = submission['Decision'][-1] + diff



submission

,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Decision
ABBV,0.00,0.04,0.20,0.42,0.34,0.01697
ACN,1.00,0.00,0.00,0.00,0.00,0.00000
AEP,0.00,0.00,0.00,0.00,1.00,0.04990
AIZ,0.00,0.05,0.13,0.51,0.31,0.01547
ALLE,0.00,0.03,0.08,0.38,0.51,0.02545
AMAT,0.38,0.41,0.10,0.10,0.01,0.00050
AMP,0.16,0.40,0.28,0.16,0.00,0.00000
AMZN,1.00,0.00,0.00,0.00,0.00,0.00000
AVB,0.03,0.24,0.40,0.29,0.04,0.00200
AVY,0.00,0.00,0.00,0.02,0.98,0.04890


In [73]:
# Save to file
submission.to_csv('submission.csv')
submission

,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Decision
ABBV,0.00,0.04,0.20,0.42,0.34,0.01697
ACN,1.00,0.00,0.00,0.00,0.00,0.00000
AEP,0.00,0.00,0.00,0.00,1.00,0.04990
AIZ,0.00,0.05,0.13,0.51,0.31,0.01547
ALLE,0.00,0.03,0.08,0.38,0.51,0.02545
AMAT,0.38,0.41,0.10,0.10,0.01,0.00050
AMP,0.16,0.40,0.28,0.16,0.00,0.00000
AMZN,1.00,0.00,0.00,0.00,0.00,0.00000
AVB,0.03,0.24,0.40,0.29,0.04,0.00200
AVY,0.00,0.00,0.00,0.02,0.98,0.04890
